In [1]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import CharacterTextSplitter

# Data Ingestion

In [6]:
loader = TextLoader("speech.txt")
documents = loader.load()

In [7]:
documents

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness be

# Data transformation

In [12]:
text_splitter = CharacterTextSplitter(chunk_size=1000,chunk_overlap=30)
docs = text_splitter.split_documents(documents)

In [13]:
docs

[Document(metadata={'source': 'speech.txt'}, page_content='The world must be made safe for democracy. Its peace must be planted upon the tested foundations of political liberty. We have no selfish ends to serve. We desire no conquest, no dominion. We seek no indemnities for ourselves, no material compensation for the sacrifices we shall freely make. We are but one of the champions of the rights of mankind. We shall be satisfied when those rights have been made as secure as the faith and the freedom of nations can make them.\n\nJust because we fight without rancor and without selfish object, seeking nothing for ourselves but what we shall wish to share with all free peoples, we shall, I feel confident, conduct our operations as belligerents without passion and ourselves observe with proud punctilio the principles of right and of fair play we profess to be fighting for.\n\n…'),
 Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct our

# Embedding

In [16]:
embeddings = OllamaEmbeddings(model='gemma:2b') 

# VectorStore

In [17]:
db=FAISS.from_documents(docs,embeddings)

In [ ]:
# Querying -> pour tester
query = "How does the speaker describe the desired outcome of the war"

result = db.similarity_search(query)
result[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

In [26]:
# As a retriever -> pour un RAG
retriever = db.as_retriever()
docs = retriever.invoke(query)
docs[0].page_content

'…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'

retriever est fait pour être branché dans un pipeline RAG,

In [27]:
# Similarity Search with score : score L2

docs_with_score = db.similarity_search_with_score(query)
docs_with_score

[(Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
  4990.3613),
 (Document(metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We s

In [30]:
embedding_vector = embeddings.embed_query(query)
embedding_vector

[-2.0188028812408447,
 0.1609557867050171,
 -0.7721268534660339,
 0.8209736347198486,
 -1.8449716567993164,
 0.5810369849205017,
 -0.8650059700012207,
 -0.4152374267578125,
 -0.002161418553441763,
 -1.7812668085098267,
 0.4064008295536041,
 0.49736568331718445,
 -0.0870913565158844,
 0.11274759471416473,
 1.8482532501220703,
 1.2526791095733643,
 2.1217010021209717,
 -0.05554155632853508,
 1.391635775566101,
 -0.29531702399253845,
 -0.6542063355445862,
 0.8628296852111816,
 2.0029969215393066,
 -0.4990738332271576,
 -1.6589547395706177,
 0.526496410369873,
 -1.2120004892349243,
 -0.9113690257072449,
 1.0113685131072998,
 -0.1187971755862236,
 -0.026646340265870094,
 0.018595242872834206,
 1.0262279510498047,
 -2.788191080093384,
 -0.9179173111915588,
 -0.7687859535217285,
 -0.2787610590457916,
 1.3589643239974976,
 0.9479327201843262,
 -0.979977011680603,
 0.8578747510910034,
 -0.3753066062927246,
 1.7555830478668213,
 -1.9839415550231934,
 -0.8510685563087463,
 0.19676868617534637,
 -

In [34]:
docs_score = db.similarity_search_by_vector(embedding_vector)
docs_score

[Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, s

In [35]:
# Saving and loading
db.save_local("faiss_index")

In [38]:
new_db=FAISS.load_local("faiss_index",embeddings,allow_dangerous_deserialization=True)  # allow_dangerous_deserialization : Confiance dans le dossier

In [40]:
docs = new_db.similarity_search(query)
docs

[Document(metadata={'source': 'speech.txt'}, page_content='…\n\nIt will be all the easier for us to conduct ourselves as belligerents in a high spirit of right and fairness because we act without animus, not in enmity toward a people or with the desire to bring any injury or disadvantage upon them, but only in armed opposition to an irresponsible government which has thrown aside all considerations of humanity and of right and is running amuck. We are, let me say again, the sincere friends of the German people, and shall desire nothing so much as the early reestablishment of intimate relations of mutual advantage between us—however hard it may be for them, for the time being, to believe that this is spoken from our hearts.'),
 Document(metadata={'source': 'speech.txt'}, page_content='We have borne with their present government through all these bitter months because of that friendship—exercising a patience and forbearance which would otherwise have been impossible. We shall, happily, s